In [16]:
import os, os.path

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import time
import random
import numpy as np
import math
import matplotlib
import typing

In [17]:
def get_activation_function(activation_fn_name: str):
    """Convert from an activation function name to the function itself."""
    if activation_fn_name is None:
        return None
    activation_fn_name = activation_fn_name.lower()

    string_to_activation_fn = {
        "linear": None,
        "tanh": tf.nn.tanh,
        "relu": tf.nn.relu,
        "leaky_relu": tf.nn.leaky_relu,
        "elu": tf.nn.elu,
        "selu": tf.nn.selu,
        "sigmoid": tf.nn.sigmoid
    }
    activation_fn = string_to_activation_fn.get(activation_fn_name)
    if activation_fn is None:
        raise ValueError(f"Unknown activation function: {activation_fn_name}")
    return activation_fn

In [18]:
def degree_matrix(matrix):
    n = matrix.shape[0]
    degree_m = np.zeros((n, n))
    for i in range(n):
        degree_m[i][i] = matrix[i].sum()
    return degree_m


In [19]:
"""
Basic Graph Convolutional layer
"""

class GCNLayer(layers.Layer):
    def __init__(self, output_dim, activation="linear", **kwargs):
        self.output_dim = output_dim
        self.activation = get_activation_function(activation)
        super(GCNLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        A_shape, H_shape = input_shape
        self.w = self.add_weight(name="weights", shape=[H_shape[-1], self.output_dim])
        self.b = self.add_weight(name="bias", shape=[self.output_dim])

        super(GCNLayer, self).build(input_shape)

    def call(self, inputs, **kwargs):
        print(inputs)
        A, H = inputs[0], inputs[1]
        if self.activation != "None":
            return self.activation(tf.matmul(tf.matmul(A, H), self.w) + self.b)
        return (tf.matmul(tf.matmul(A, H), self.w) + self.b)

        


In [20]:
"""
Gated Graph Cell
Input is a 2n x n adjacency matrix in addition to the initial embedding. 2n as in the paper they split up edges 
in non directed graphs into two directed edges. Might just add calculating the second matrix into the call part of the 
fn so that we dont need to write out seperate adjacency matrices 


propogation_no is the number of propogation steps
Hopefull that this is the right implementation

"""

class GGNNcell(layers.Layer):
    def __init__(self, propogation_no, activation="sigmoid", **kwargs):
        self.activation = get_activation_function(activation)
        self.propogation_no = propogation_no
        super(GGNNcell, self).__init__(**kwargs)

    def build(self, input_shape):
        A_shape, H_shape = input_shape

        self.bias = self.add_weight(name="bias", shape=[A_shape[-1], H_shape[-1]])

        self.w_reset = self.add_weight(name="w_reset", shape=[A_shape[-1], A_shape[-2]])
        self.u_reset = self.add_weight(name="u_reset", shape=[A_shape[-1], A_shape[-1]])

        self.w_update = self.add_weight(name="w_update", shape=[A_shape[-1], A_shape[-2]])
        self.u_update = self.add_weight(name="u_update", shape=[A_shape[-1], A_shape[-1]])

        self.w = self.add_weight(name="w", shape=[A_shape[-1], A_shape[-2]])
        self.u = self.add_weight(name="u", shape=[A_shape[-1], A_shape[-1]])

    def call(self, inputs):
        A, H = inputs[0], inputs[1]
        initial_code = inputs[1]
        for i in range(self.propogation_no):
            a = tf.matmul(A, H) + self.bias

            reset = self.activation(tf.matmul(self.w_reset, a) + tf.matmul(self.u_reset, H))
            update = self.activation(tf.matmul(self.w_update, a) + tf.matmul(self.u_update, H))

            h_update = tf.nn.tanh(tf.matmul(self.w, a) + tf.matmul(self.u, (tf.math.multiply(reset, H))))
            h_update = tf.math.multiply(1 - update, H) + tf.math.multiply(update, h_update)

            H = h_update

        return A, H

In [21]:
"""
Graph level output which involves two networks taking in both initial embeddings and propogated embeddings
then taking the dot product per vertex of the two networks output, then summing them together and finally applying 
some sigmoid fn. Dot product apparently is acting as a sort of attention mechanism, but will read up on that. 
"""

"still need to implement the node level output"

class GGNN_Graph_out(layers.Layer):
    def __init__(self, propogatino_no, output_dim, **kwargs):
        self.GGNN_cell = GGNNcell(propogatino_no)
        self.network_i = layers.Dense(output_dim, actvation = 'sigmoid ')
        self.network_j = layers.Dense(output_dim, activation = 'tanh')
        super(GGNN_Graph_out, self).__init__(**kwargs)


    def call(self, inputs):
        A, H = inputs[0], inputs[1]
        prop_out = self.GGNN_cell(A, H)[1]
        a = self.network_i((prop_out, H))
        b = self.network_j((prop_out, H))
        dot = tf.math.multiply(a,b)
        dot = tf.nn.tanh(tf.math.reduce_sum(dot, axis = 0))
        return dot


In [22]:
graph_layer = GGNNcell(1)
graph_layer = GCNLayer(2, activation="tanh")
graph = np.asarray([[1, 1, 1], [1, 1, 0], [1, 0, 1]]).astype('float32')
embedding = np.asarray([[1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0]]).astype('float32')

print(graph_layer((graph, embedding)))

(<tf.Tensor: id=36, shape=(3, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 0.],
       [1., 0., 1.]], dtype=float32)>, <tf.Tensor: id=37, shape=(3, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)>)
tf.Tensor(
[[-0.9987618  -0.9879896 ]
 [-0.9922957  -0.93105525]
 [-0.9922957  -0.93105525]], shape=(3, 2), dtype=float32)
